### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

## Loading and Analysing Dataframe

In [ ]:
df = pd.read_csv('../data/train-sample.csv')

In [ ]:
print(df.columns)
df.head(3)

### Features To Create/Encode
From these columns, I consider important:
- Title,
- BodyMarkdown,
- SelectedTags (top N tags, rest as "other"),
- user life at creation (PostCreationDate - OwnerCreationDate)
- OpenStatus

I will create them in the next sections

## Data processing

### Statuses

In [ ]:
statuses = df['OpenStatus'].unique()
statuses

### Most Frequent Tags

In [ ]:
tag_column_names = ['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']
tags_long = list(df[tag_column_names].values.ravel('K'))

tags_unique, frequencies = np.unique(tags_long, return_counts=True)
freq_dict = {tags_unique[i]: frequencies[i] for i in range(len(tags_unique))}

In [ ]:
tags_freq_arr = sorted(freq_dict.items(), key=lambda kv: -1 * kv[1])

print('there is {} unique tags, some of them (along with frequencies): {}'.format(len(tags_freq_arr), tags_freq_arr[:10]))

As we can see, the most frequent value is 'nan' - we do not want to include it so it will be removed in the following cells.

We will also select N the most frequently occuring tags and classify the rest as `other`

In [ ]:
N_tags = 500

tags_freq_arr = tags_freq_arr[1:][:N_tags]

selected_tags = np.empty(shape=N_tags, dtype=tags_unique.dtype)
for i, item in enumerate(tags_freq_arr):
  selected_tags[i] = item[0]

### Selecting Best Features

In [ ]:
print(df.columns)
df.head(3)

In [ ]:
df['DaysTillPosting'] = (df['PostCreationDate'] - df['OwnerCreationDate']).dt.days

In [ ]:
print(df.columns)
df.head(3)

## Data processing

### Most Frequent Tags

In [ ]:
N_tags = 500

tags_freq_arr = tags_freq_arr[1:][:N_tags]

selected_tags = np.empty(shape=N_tags, dtype=tags_unique.dtype)
for i, item in enumerate(tags_freq_arr):
  selected_tags[i] = item[0]

### Days Since Account Creation Till Posting Question

In [ ]:
df['OwnerCreationDate'] = pd.to_datetime(df['OwnerCreationDate'], format='mixed')
df['PostCreationDate'] = pd.to_datetime(df['PostCreationDate'], format='mixed')

In [ ]:
df['DaysTillPosting'] = (df['PostCreationDate'] - df['OwnerCreationDate']).dt.days

### Processing Tags

In [ ]:
def getRecognizedTags(row):
  recognized_tags = []
  
  for tag in row[tag_column_names].values:
    if(np.isin(tag, selected_tags)):
      recognized_tags.append(tag)
  
  return recognized_tags

def getUnrecognizedTags(row):
  unrecognized_tags = 0
  
  for tag in row[tag_column_names].values:
    if(~np.isin(tag, selected_tags)):
      unrecognized_tags += 1
  
  return unrecognized_tags

In [ ]:
df["RecognizedTags"] = df[:1000].apply(getRecognizedTags, axis=1)
df["UnrecognizedTags"] = df[:1000].apply(getUnrecognizedTags, axis=1)

# REMOVE [:1000] !!!